In [304]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas

In [313]:
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0',
}
first_pg_hh='https://www.superjob.ru/vakansii/shveya.html?geo%5Bt%5D%5B0%5D=4'

In [347]:
def total_parsing_hh(first_pg, headers, result=[]):

    response = requests.get(first_pg, headers=headers) 
    
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    soup = bs(response.content, 'html.parser')
    tag_main_div=soup.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in tag_main_div:
        result.append(vac_parsing(vacancy))
    
    link_next_page = soup.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
        link_next_page =  'https://hh.ru' + link_next_page['href']
        
    else:
        print('Парсинг завершен')
        return result
    result=total_parsing_hh(link_next_page, headers,result)                        
    return result
    

In [346]:
hh_vcncy_list=[]
def salary_split(tag_span_salary, vac_min_salary=None, vac_max_salary=None, currency=None):
    sleep(0.1)
    tag_span_salary=tag_span_salary.text.replace('\u202f','').split()   
    if tag_span_salary[1]=='–':
        vac_max_salary=tag_span_salary[2]
        vac_min_salary=tag_span_salary[0]       
    elif  tag_span_salary[0]=='до':
        vac_max_salary=tag_span_salary[1]        
    elif tag_span_salary[0]=='от':
        vac_min_salary=tag_span_salary[1]    
    currency=tag_span_salary[-1]
        
    
    return vac_min_salary, vac_max_salary, currency
def vac_parsing(tag_main_div):
    sleep(0.1)

    tag_a_name=tag_main_div.find('a').text.strip()

    tag_span_salary=tag_main_div.find('span', {'class', 'bloko-header-section-3'})
    if tag_span_salary:
          vac_min_salary, vac_max_salary, currency=salary_split(tag_span_salary)
    else:
        vac_min_salary, vac_max_salary, currency=None,None,None
    tag_a_vcnc_link=tag_main_div.find('a')['href']
    
    return {
        'name': tag_a_name,
        'min salary': vac_min_salary,
        'max salary': vac_max_salary,
        'currency': currency,
        'vac_link': tag_a_vcnc_link,
        'site':'https://hh.ru'
        }

In [348]:
result=total_parsing_hh(first_pg_hh,headers)

Парсинг завершен


In [350]:
len(result)

380

In [351]:
result[0]

{'name': 'Швея/портной-универсал',
 'min salary': '60000',
 'max salary': '110000',
 'currency': 'руб.',
 'vac_link': 'https://hh.ru/vacancy/69675667?query=%D1%88%D0%B2%D0%B5%D1%8F',
 'site': 'https://hh.ru'}

In [352]:
data_frame_hh = pandas.DataFrame(result) 

data_frame_hh.to_csv('hh.csv') 

In [233]:
soup = bs(response.content, 'html.parser')

In [239]:
tag_main_div=soup.find_all('div', {'class': 'vacancy-serp-item__layout'})

len(tag_main_div)

50

In [265]:
tag_a_name=tag_main_div[3].find('a').text
tag_a_name

'Швея ученик'

In [287]:
tag_span_salary=tag_main_div[0].find('span', {'class', 'bloko-header-section-3'}).text
tag_span_salary.replace('\u202f','').split()

['60000', '–', '120000', 'руб.']

In [267]:
tag_a_vcnc_link=tag_main_div[3].find('a')['href']
tag_a_vcnc_link

'https://hh.ru/vacancy/69488501?query=%D1%88%D0%B2%D0%B5%D1%8F'